# MIT License

Copyright (c) 2021 Priyanka Dilip Shinde

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# Abstract
About Data: The training dataset contains X-ray and CT scan images of patients with positive Corona test and negative Corona test. 

Aim: To build a CNN model that will analyze and detect if the patient is a COVID (i.e. the target value as 1) patient or Non-COVID (i.e. the target value as 0) patient.

Loaded Libraries and dataset, loaded image file path, confirmed if number of labels is equal to number of images, converted the file paths to dataframe, combined labels with images for classification, pre-processed image dataset, separated images from labels, divied the data into training and testing dataset, sclaed the data, build CNN model and performed hyperparameter tunning to compare accuracy and check how the CNN network plateaus, used Tensorboard to visualize the result.

# Loading Libraries

In [1]:
from sklearn.model_selection import cross_val_score
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Flatten
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import make_scorer, accuracy_score
from tensorflow.keras.layers import Dense,Conv2D
from tensorflow.keras import initializers

from keras.wrappers.scikit_learn import KerasClassifier
from keras_tuner import RandomSearch
import datetime
from torch.utils.tensorboard import SummaryWriter
from tensorflow.keras.callbacks import TensorBoard

import pandas as pd
import numpy as np
import tensorflow as tf                                 
import os                                               
import cv2                                              
from sklearn.model_selection import train_test_split    
from sklearn.metrics import f1_score
import torch

Using TensorFlow backend.


In [2]:
%load_ext tensorboard

In [3]:
train = pd.read_csv("Training_set_covid.csv")

In [4]:
train

,filename,label
0,Image_1.jpg,1
1,Image_2.jpg,0
2,Image_3.jpg,0
3,Image_4.jpg,0
4,Image_5.jpg,0
...,...,...
3474,Image_3475.jpg,0
3475,Image_3476.jpg,0
3476,Image_3477.jpg,0
3477,Image_3478.jpg,1


# Getting images file path

In [5]:
file_paths = [[fname, '/Users/priyankashinde/Downloads/covid_image_data/train/' + fname] for fname in train['filename']]

In [6]:
file_paths

[['Image_1.jpg',
  '/Users/priyankashinde/Downloads/covid_image_data/train/Image_1.jpg'],
 ['Image_2.jpg',
  '/Users/priyankashinde/Downloads/covid_image_data/train/Image_2.jpg'],
 ['Image_3.jpg',
  '/Users/priyankashinde/Downloads/covid_image_data/train/Image_3.jpg'],
 ['Image_4.jpg',
  '/Users/priyankashinde/Downloads/covid_image_data/train/Image_4.jpg'],
 ['Image_5.jpg',
  '/Users/priyankashinde/Downloads/covid_image_data/train/Image_5.jpg'],
 ['Image_6.jpg',
  '/Users/priyankashinde/Downloads/covid_image_data/train/Image_6.jpg'],
 ['Image_7.jpg',
  '/Users/priyankashinde/Downloads/covid_image_data/train/Image_7.jpg'],
 ['Image_8.jpg',
  '/Users/priyankashinde/Downloads/covid_image_data/train/Image_8.jpg'],
 ['Image_9.jpg',
  '/Users/priyankashinde/Downloads/covid_image_data/train/Image_9.jpg'],
 ['Image_10.jpg',
  '/Users/priyankashinde/Downloads/covid_image_data/train/Image_10.jpg'],
 ['Image_11.jpg',
  '/Users/priyankashinde/Downloads/covid_image_data/train/Image_11.jpg'],
 ['Ima

# Confirming if no. of labels is equal to no. of images

In [7]:
if len(train) == len(file_paths):
    print('Number of labels i.e. ', len(train), 'matches the number of filenames i.e. ', len(file_paths))
else:
    print('Number of labels does not match the number of filenames')

Number of labels i.e.  3479 matches the number of filenames i.e.  3479


# Converting the file_paths to dataframe

In [8]:
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
images.head()

,filename,filepaths
0,Image_1.jpg,/Users/priyankashinde/Downloads/covid_image_da...
1,Image_2.jpg,/Users/priyankashinde/Downloads/covid_image_da...
2,Image_3.jpg,/Users/priyankashinde/Downloads/covid_image_da...
3,Image_4.jpg,/Users/priyankashinde/Downloads/covid_image_da...
4,Image_5.jpg,/Users/priyankashinde/Downloads/covid_image_da...


# Combining the labels with the images

In [9]:
train_data = pd.merge(images, train, how = 'inner', on = 'filename')
train_data.head()

,filename,filepaths,label
0,Image_1.jpg,/Users/priyankashinde/Downloads/covid_image_da...,1
1,Image_2.jpg,/Users/priyankashinde/Downloads/covid_image_da...,0
2,Image_3.jpg,/Users/priyankashinde/Downloads/covid_image_da...,0
3,Image_4.jpg,/Users/priyankashinde/Downloads/covid_image_da...,0
4,Image_5.jpg,/Users/priyankashinde/Downloads/covid_image_da...,0


In [10]:
type(train_data)

pandas.core.frame.DataFrame

In [11]:
x = train_data.filepaths
y = train_data.label

# Data Pre-processing

It is necessary to bring all the images in the same shape and size, also convert them to their pixel values because all machine learning or deep learning models accepts only the numerical data. Also we need to convert all the labels from categorical to numerical values.

In [12]:
data = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):
  
    img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

    new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
    data.append([new_img_array, train_data['label'][i]])

# Separating the images and labels

In [13]:
x = []
y = []
for image in data:
    x.append(image[0])
    y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y)

# Splitting the data into Train and Validation Set

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state = 42)

# Scaling the dataset

In [15]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [16]:
x_train.shape

(2435, 100, 100)

In [17]:
x_train = x_train.reshape(len(x_train),100,100,1)

In [18]:
x_test = x_test.reshape(len(x_test),100,100,1)

# Activation Function 

In [19]:
log_folder = 'logs'

In [20]:
log_folder = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [21]:
tensorboard_callback = [TensorBoard(log_dir=log_folder,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,
                         embeddings_freq=1)]

In [22]:
def build_model(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(100,100,1)),
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    # output layer    
    keras.layers.Dense(10, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [23]:
tuner = RandomSearch(build_model,
                    objective='val_accuracy',
                    max_trials = 1,
                    directory='tuner')

In [24]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [25]:
tuner.search(x=x_train, y=y_train, epochs=4, callbacks=tensorboard_callback, validation_data=(x_train, y_train))

Trial 1 Complete [00h 02m 31s]
val_accuracy: 0.9901437163352966

Best val_accuracy So Far: 0.9901437163352966
Total elapsed time: 00h 02m 31s
INFO:tensorflow:Oracle triggered exit


In [26]:
tuner.results_summary()

Results summary
Results in tuner/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 3
dense_1_units: 80
learning_rate: 0.001
Score: 0.9901437163352966


In [4]:
%tensorboard --logdir logs

In [28]:
model=tuner.get_best_models(num_models=1)[0]
#summary of best model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 32)        320       
_________________________________________________________________
flatten (Flatten)            (None, 307328)            0         
_________________________________________________________________
dense (Dense)                (None, 80)                24586320  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 24,587,450
Trainable params: 24,587,450
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.fit(x_test,y_test,
          epochs=4,
          validation_split=0.1, initial_epoch=1)

Train on 939 samples, validate on 105 samples
Epoch 2/4
939/939 [==============================] - 9s 10ms/sample - loss: 0.2011 - accuracy: 0.9127 - val_loss: 0.1164 - val_accuracy: 0.9429
Epoch 3/4
939/939 [==============================] - 9s 10ms/sample - loss: 0.0905 - accuracy: 0.9563 - val_loss: 0.1587 - val_accuracy: 0.9238
Epoch 4/4
939/939 [==============================] - 10s 10ms/sample - loss: 0.0520 - accuracy: 0.9787 - val_loss: 0.1307 - val_accuracy: 0.9429


## sigmoid

In [30]:
log_folder1 = 'logs1'

In [31]:
log_folder1 = "logs1/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [32]:
tensorboard_callback1 = [TensorBoard(log_dir=log_folder1,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,            
                         embeddings_freq=1)]

In [33]:
def build_model1(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='sigmoid',
        input_shape=(100,100,1)),
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='sigmoid'
    ),
    # output layer    
    keras.layers.Dense(10, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [34]:
tuner = RandomSearch(build_model1,
                    objective='val_accuracy',
                    max_trials = 1,
                     directory='tuner1')

In [35]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [36]:
tuner.search(x=x_train, y=y_train, epochs=4, callbacks=tensorboard_callback1, validation_data=(x_train, y_train))

Trial 1 Complete [00h 02m 57s]
val_accuracy: 0.8825461864471436

Best val_accuracy So Far: 0.8825461864471436
Total elapsed time: 00h 02m 57s
INFO:tensorflow:Oracle triggered exit


In [37]:
tuner.results_summary()

Results summary
Results in tuner1/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 5
dense_1_units: 48
learning_rate: 0.001
Score: 0.8825461864471436


In [5]:
%tensorboard --logdir logs1

In [39]:
model=tuner.get_best_models(num_models=1)[0]
#summary of best model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 32)        832       
_________________________________________________________________
flatten (Flatten)            (None, 294912)            0         
_________________________________________________________________
dense (Dense)                (None, 48)                14155824  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 14,157,146
Trainable params: 14,157,146
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(x_test,y_test,
          epochs=4,
          validation_split=0.1)

#### Accuracy decreases with change in activation function from relu(99%) to sigmoid(88.2%)

# Cost Function

In [41]:
log_folder2 = 'logs2'

In [42]:
log_folder2 = "logs2/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [43]:
tensorboard_callback2 = [TensorBoard(log_dir=log_folder2,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,           
                         embeddings_freq=1)]

In [44]:
def build_model2(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(100,100,1)),
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    # output layer    
    keras.layers.Dense(10, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='squared_hinge',
              metrics=['accuracy'])
    return model

In [45]:
tuner = RandomSearch(build_model2,
                    objective='val_accuracy',
                    max_trials = 1,
                    directory='tuner2')

In [46]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [47]:
tuner.search(x=x_train, y=y_train, epochs=4, callbacks=tensorboard_callback2, validation_data=(x_train, y_train))

Trial 1 Complete [00h 04m 14s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 04m 14s
INFO:tensorflow:Oracle triggered exit


In [48]:
tuner.results_summary()

Results summary
Results in tuner2/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 5
dense_1_units: 80
learning_rate: 0.001
Score: 1.0


In [6]:
%tensorboard --logdir logs2

#### Accuracy is more for squared_hinge cost function as compared to sparse_categorical_crossentropy

# Epochs

In [50]:
log_folder3 = 'logs3'

In [51]:
log_folder3 = "logs3/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [52]:
tensorboard_callback3 = [TensorBoard(log_dir=log_folder3,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,           
                         embeddings_freq=1)]

In [53]:
def build_model3(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(100,100,1)),
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    # output layer    
    keras.layers.Dense(10, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [54]:
tuner = RandomSearch(build_model3,
                    objective='val_accuracy',
                    max_trials = 1,
                     directory='tuner3')

In [55]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [56]:
tuner.search(x=x_train, y=y_train, epochs=8, callbacks=tensorboard_callback3, validation_data=(x_train, y_train))

Trial 1 Complete [00h 12m 49s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 12m 49s
INFO:tensorflow:Oracle triggered exit


In [57]:
tuner.results_summary()

Results summary
Results in tuner3/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 3
dense_1_units: 112
learning_rate: 0.001
Score: 1.0


In [7]:
%tensorboard --logdir logs3

#### Accuracy increases with number of epochs 

# Gradient Estimation

In [59]:
log_folder4 = 'logs4'

In [60]:
log_folder4 = "logs4/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [61]:
tensorboard_callback4 = [TensorBoard(log_dir=log_folder4,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,            
                         embeddings_freq=1)]

In [62]:
def build_model4(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(100,100,1)),
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    # output layer    
    keras.layers.Dense(10, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.RMSprop(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [63]:
tuner = RandomSearch(build_model4,
                    objective='val_accuracy',
                    max_trials = 1,
                     directory='tuner4')

In [64]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [65]:
tuner.search(x=x_train, y=y_train, epochs=4, callbacks=tensorboard_callback4, validation_data=(x_train, y_train))

Trial 1 Complete [00h 03m 09s]
val_accuracy: 0.9223819375038147

Best val_accuracy So Far: 0.9223819375038147
Total elapsed time: 00h 03m 09s
INFO:tensorflow:Oracle triggered exit


In [66]:
tuner.results_summary()

Results summary
Results in tuner4/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 3
dense_1_units: 112
learning_rate: 0.01
Score: 0.9223819375038147


In [8]:
%tensorboard --logdir logs4

#### Accuracy using Adam(99%) gradient estimation is more as compared to RMSprop(92%)

# Modifying Network Architecture(Adding Second Convolution Layer)

Filters: Creating a range of integers that takes a certain values

kernel_size: An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window.

In [68]:
log_folder5 = 'logs5'

In [69]:
log_folder5 = "logs5/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [70]:
tensorboard_callback5 = [TensorBoard(log_dir=log_folder5,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,
                         embeddings_freq=1)]

In [71]:
def build_model5(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(100,100,1)),
    # adding second convolutional layer 
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        #activation function
        activation='relu'
    ),
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    # output layer    
    keras.layers.Dense(10, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [72]:
tuner = RandomSearch(build_model5,
                    objective='val_accuracy',
                    max_trials = 1,
                    directory='tuner5')

In [73]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
conv_2_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 16, 'sampling': None}
conv_2_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [74]:
tuner.search(x=x_train, y=y_train, epochs=4, callbacks=tensorboard_callback5, validation_data=(x_train, y_train))

Trial 1 Complete [00h 13m 09s]
val_accuracy: 0.9655030965805054

Best val_accuracy So Far: 0.9655030965805054
Total elapsed time: 00h 13m 09s
INFO:tensorflow:Oracle triggered exit


In [75]:
tuner.results_summary()

Results summary
Results in tuner5/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 5
conv_2_filter: 64
conv_2_kernel: 3
dense_1_units: 96
learning_rate: 0.01
Score: 0.9655030965805054


In [9]:
%tensorboard --logdir logs5

#### Accuracy does not increase with the addition of second convolution layer max accuracy achieved is 96.5%

# Network Initialization 

## 'he' Initialization 

In [94]:
log_folder6 = 'logs6'

In [95]:
log_folder6 = "logs6/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [96]:
tensorboard_callback6 = [TensorBoard(log_dir=log_folder6,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,
                         embeddings_freq=1)]

In [99]:
def build_model6(hp):
    # create model object
    model = keras.Sequential([
    
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(100,100,1)),
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    # output layer    
    keras.layers.Dense(10, kernel_initializer=initializers.he_uniform(), activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [100]:
tuner = RandomSearch(build_model6,
                    objective='val_accuracy',
                    max_trials = 1,
                    directory='tuner7')

In [101]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [102]:
tuner.search(x=x_train, y=y_train, epochs=4, callbacks=tensorboard_callback6, validation_data=(x_train, y_train))

Trial 1 Complete [00h 03m 09s]
val_accuracy: 0.9724845886230469

Best val_accuracy So Far: 0.9724845886230469
Total elapsed time: 00h 03m 09s
INFO:tensorflow:Oracle triggered exit


In [103]:
tuner.results_summary()

Results summary
Results in tuner7/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 3
dense_1_units: 48
learning_rate: 0.01
Score: 0.9724845886230469


In [10]:
%tensorboard --logdir logs6

#### Accuracy does not increase after network initialization max accuracy achieved is 97% 

# References

1. https://www.tensorflow.org/tensorboard/image_summaries#building_an_image_classifier

2. https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams

3. https://machinelearningmastery.com/weight-initialization-for-deep-learning-neural-networks/

# Conclusion

Acieved Best val_accuracy So Far: 1 with following parameter:

conv_1_filter: 48

conv_1_kernel: 5

dense_1_units: 80

learning_rate: 0.0001

activation function: Adam

cost function: squared_hinge

number of epochs: 4

gradient estimator: relu and softmax